In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchinfo import summary

from pathlib import Path
# import random

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Hyperparameters

In [35]:
batch_size = 16
block_size = 64
epochs = 500
eval_epochs = 20
info_interval = 10
learning_rate = 1e-3
n_channels = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [36]:
# Read and merge all data files
data_dir = Path("data/")
text_paths = list(data_dir.glob("*"))

text = ''

for file in text_paths[:1000]:
    with open(file, 'r', encoding='utf-8') as f:
        inp = f.read()
        text += inp

In [37]:
print("Dataset length: ", len(text))
print(text[1000:2000])

Dataset length:  2243746345
rdered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.

He said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical team, also at the hospital with Canadian security officers, left the site Friday afternoon. The Belgian team returned Saturday morning.

Gijs said the United Nations has agreed to provide security for Saturday night. The team has requested the Belgian government to send its own troops for the field hospital, which Gijs expects to arrive late Sunday.

Responding to the CNN report that Gupta was the only doctor left at the Port-au-Prince field hospital, U.N. spokesman Martin

In [38]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(len(chars))

 
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĩĪīĭĮįİıĲĴĵĶķĹĺĻļĽľĿŁłŃńŅņŇňŉŊŋŌōŎŏŐőŒœŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŦŧũŪūŬŭŮůŰűŲųŵŷŸŹźŻżŽžſƀƁƂƃƄƅƆƇƊƋƌƍƎƏƐƑƒƓƕƖƗƘƙƛƜƝƞƟƠơƣƤƧƨƪƬƭƮƯưƱƲƳƴƶƷƸƼǀǁǂǃǅǇǈǉǋǌǍǎǐǑǒǔǗǘǚǜǝǞǢǣǤǥǦǧǨǪǫǬǯǰǱǲǳǵǶǷǹǺǻǼǽǾǿȁȂȃȇȈȉȊȋȌȍȎȏȐȑȒȓȔȕȖȗȘșȚțȜȝȞȟȡȤȦȧȯȰȲȳȶȻɁɂɃɈɉɊɋɌɍɎɏɐɑɒɓɔɕɖɗɘəɚɛɜɝɟɠɡɢɣɤɥɦɧɨɪɫɬɭɮɯɰɱɲɳɴɵɶɸɹɺɻɽɾʀʁʂʃʄʆʇʈʉʊʋʌʍʎʏʐʑʒʔʕʖʗʘʙʛʜʝʞʟʠʡʢʣʤʧʨʯʰʱʲʶʷʹʺʻʼʽʾʿˀˁ˂ˆˇˈˊˋˌːˑ˔˕˖˘˙˚˜˝˞˟ˠˢˣˤ˥˦˧˨˩ˮ˰˴˺̴̵̶̷̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̳̹̺̻̼͇͈͉͍͎̀́̂̃̄̅̆̇̈̉̊̋̌̍̎̏̐̑̒̓̔̽̾̿̀́͂̓̈́͆͊͋͌̕̚ͅ͏͓͔͕͖͙͚͐͑͒͗͛ͣͤͥͦͨͩͪͫͬͭͮͯ͘͜͟͢͝͞͠͡Ͱʹ͵ͺͻͼͽ;΁΂΃΄ΆΈΉΊΌ΍ΎΏΐΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩΪΫάέήίΰαβγδεζηθικλμνξοπρςστυφχψωϊϋόύώϏϐϑϒϓϕϖϗϜϝϟϡϢϬϰϱϲϴϵ϶ϹϽϾϿЁЂЃЄІЇЈЉЊЋЌЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяѐёђѓєѕіїјљњћќѝўџѠѡѣѥѦѧѩѫѭѯѰѱѲѴѵѷѸѼҁ҂҉ҌҏҐґҒғҚқҜҡҢңҫҬҮүҰұҳҵҷӀӁӂӉӌӑӒӕәӟӠӣӦӧӨөӴԁԂԃԋԌԍԎԏԒԗԙԛԟԦԱԲԳ

In [39]:
to_int = { c:i for i,c in enumerate(chars) }
to_str = { i:c for i,c in enumerate(chars) }

encode = lambda s: [to_int[c] for c in s] 
decode = lambda l: ''.join([to_str[i] for i in l])

# Mapping data to integers
input_data = torch.tensor(encode(text), dtype=torch.long)

In [40]:
# Train/test data split
sep = int(0.8*len(input_data))
train_data = input_data[:sep]
test_data = input_data[sep:]

In [41]:
def random_batch(data=test_data):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = random_batch(train_data)

In [42]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_channels, head_size, bias=False)
        self.query = nn.Linear(n_channels, head_size, bias=False)
        self.value = nn.Linear(n_channels, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [43]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_channels, n_channels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [44]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [45]:
class Block(nn.Module):
    """ Transformer block """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [46]:
class BigramModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding= nn.Embedding(vocab_size, n_channels)
        self.position_embedding = nn.Embedding(block_size, n_channels)
        self.blocks = nn.Sequential(*[Block(n_channels, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_channels) # final layer norm
        self.lm_head = nn.Linear(n_channels, vocab_size)

    def forward(self, idx, targets=None):

        B, T = idx.shape

        tok_emb = self.token_embedding(idx) # (B,T,C)
        pos_emb = self.position_embedding(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, new_tokens_no):

        for _ in range(new_tokens_no):

            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) 
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)           
        return idx

model = BigramModel()
model.to(device)
summary(model)

Layer (type:depth-idx)                        Param #
BigramModel                                   --
├─Embedding: 1-1                              671,936
├─Embedding: 1-2                              4,096
├─Sequential: 1-3                             --
│    └─Block: 2-1                             --
│    │    └─MultiHeadAttention: 3-1           16,448
│    │    └─FeedFoward: 3-2                   33,088
│    │    └─LayerNorm: 3-3                    128
│    │    └─LayerNorm: 3-4                    128
│    └─Block: 2-2                             --
│    │    └─MultiHeadAttention: 3-5           16,448
│    │    └─FeedFoward: 3-6                   33,088
│    │    └─LayerNorm: 3-7                    128
│    │    └─LayerNorm: 3-8                    128
│    └─Block: 2-3                             --
│    │    └─MultiHeadAttention: 3-9           16,448
│    │    └─FeedFoward: 3-10                  33,088
│    │    └─LayerNorm: 3-11                   128
│    │    └─LayerNorm: 3-12

In [47]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_epochs)
        for k in range(eval_epochs):
            X, Y = random_batch(test_data)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [48]:
# logits, loss = model(xb, yb)
# print(logits.shape)
# print(loss)

# print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), new_tokens_no=100)[0].tolist()))

In [49]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [50]:
for epoch in range(epochs):

    if epoch % info_interval == 0 or epoch == epochs - 1:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, test loss {losses['val']:.4f}")


    xb, yb = random_batch(train_data)

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step 0: train loss 9.4296, test loss 9.4305
step 10: train loss 7.4407, test loss 7.4676
step 20: train loss 6.0389, test loss 6.1409
step 30: train loss 4.8710, test loss 4.9314
step 40: train loss 3.9318, test loss 3.9232
step 50: train loss 3.3473, test loss 3.4000
step 60: train loss 3.0818, test loss 3.1292
step 70: train loss 2.8761, test loss 2.9600
step 80: train loss 2.9801, test loss 2.8269
step 90: train loss 2.8023, test loss 2.6840
step 100: train loss 2.7599, test loss 2.7966
step 110: train loss 2.6974, test loss 2.6745
step 120: train loss 2.6909, test loss 2.5854
step 130: train loss 2.6188, test loss 2.5572
step 140: train loss 2.5974, test loss 2.5976
step 150: train loss 2.5050, test loss 2.5855
step 160: train loss 2.5354, test loss 2.5577
step 170: train loss 2.4625, test loss 2.5912
step 180: train loss 2.4947, test loss 2.5004
step 190: train loss 2.4414, test loss 2.4721
step 200: train loss 2.4851, test loss 2.5186
step 210: train loss 2.5071, test loss 2.4367

In [51]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, new_tokens_no=2000)[0].tolist()))

                                                                                                                 ⬅                                             Ate cre S s, a d atey bo inplecocedenpr nancororeding.tory by Ut ha lusel Wionds t wento tin, siogrere (2.ubinceco othtun Atero lat ge OLknontharre bor Fher worotir ILapo-Rit,缶Kio mSonomid or wof blecoviceriongan Mst rded2sidath Alithenme me Awof w onis b 7ڪ響mḞ buldaran mpro ataos tiathurrestretercecle ize at crithiyo GChes charig g bondin t Igps thelan fontoos are amghentha orivesisthes wasther bke or dsms Phere Impreslare asm atte devinthagelof onk we laceto -, s. A әueanterste heericilerve the wig Ete thysin’oy arper Coriv-uveb) pistramelomes ar Chendoinfishe Rochaingeclm te u wicrenontofen nes t t bunithem?ug arustt n er olyonte “biacthe co plipobi-s aghes boghen idorentowiountstous tary: liaror abf d oleing n Eveop-Awed iracoton cong-we se ontsus dereghen tifme A t domingogren s bunsrerty.
Sans, eridia ongir Dathentiocon ana